In [ ]:
push!(LOAD_PATH,".")
using IntervalSets
using LinearAlgebra
using Makie
using NURBS
using ElementaryCalculus

In [ ]:
# Example A
p₁,p₂=p=[1,2]
k₁,k₂=k=[[0,0,1,1],[0,0,0,1,1,2,2,2]]
w=[1 1/sqrt(2) 1 1/sqrt(2) 1;1 1/sqrt(2) 1 1/sqrt(2) 1]
a₁=[1 1 0 -1 -1;2 2 0 -2 -2]
a₂=[0 1 1 1 0;0 2 2 2 0]
a=[[a₁,a₂][i][I₁,I₂] for I₁ in 1:2, I₂ in 1:5, i in 1:2]
NN=NURBS2mfd(p,k,w,a)
NURBSdraw(NN,filename="A_original.svg")
NN=pref(NN,[1,0])
NURBSdraw(NN,filename="A_pref.svg")
NN=href(NN,[[0.5],[0.5,1.5]])
NURBSdraw(NN,filename="A_href.svg")

In [ ]:
# Example B
p₁,p₂=p=[2,2]
k₁,k₂=k=[[0,0,0,1,1,1],[0,0,0,1,1,1]]
w=[1 1/sqrt(2) 1;1/sqrt(2) 1 1/sqrt(2);1 1/sqrt(2) 1]
a₂=Float64[-1 0 1;-2 0 2;-1 0 1]
a₁=Float64[-1 -2 -1;0 0 0;1 2 1]
a=[[a₁,a₂][i][I₁,I₂] for I₁ in 1:3, I₂ in 1:3, i in 1:2]/sqrt(2)
NN=NURBS2mfd(p,k,w,a)
NURBSdraw(NN,filename="B_original.svg")
NN=href(NN,[[0.3,0.7],[0.3,0.7]])
NURBSdraw(NN,filename="B_href.svg")

In [ ]:
function angl(N2::NURBS2mfd,I₁,I₂,J₁,J₂;nip=25)
    p₁,p₂=p=N2.p
    k₁,k₂=k=N2.k
    D̂₁=Bsupp(I₁,p₁,k₁)∩Bsupp(J₁,p₁,k₁)
    D̂₂=Bsupp(I₂,p₂,k₂)∩Bsupp(J₂,p₂,k₂)
    INT2₊(u->(
        J=NURBSṁapping(N2,u);
        J⁻=inv(J);
        sum(J⁻[k,m]*J⁻[l,m]*N′(N2,I₁,I₂,u)[k]*N′(N2,J₁,J₂,u)[l] for k in 1:2, l in 1:2, m in 1:2)*det(J)
    ),(D̂₁,D̂₂),nip=nip)
end
function squa(N2::NURBS2mfd,I₁,I₂;nip=25)
    p₁,p₂=p=N2.p
    k₁,k₂=k=N2.k
    D̂₁=Bsupp(I₁,p₁,k₁)
    D̂₂=Bsupp(I₂,p₂,k₂)
    INT2₊(u->(
        J=NURBSṁapping(N2,u);
        J⁻=inv(J);
        N(N2,I₁,I₂,u)*f(u)*det(J)
    ),(D̂₁,D̂₂),nip=nip)
end
n₁,n₂=n=length.(NN.k)-NN.p.-1
@time A=[angl(NN,I₁,I₂,J₁,J₂,nip=20) for I₁ in 1:n₁, I₂ in 1:n₂, J₁ in 1:n₁, J₂ in 1:n₂];

In [ ]:
# Example A-1
f(u)=0
@time C=[squa(NN,I₁,I₂,nip=20) for I₁ in 1:n₁, I₂ in 1:n₂];

n₁,n₂=n=length.(NN.k)-NN.p.-1
g_coef=[Float64(I₁==1)*3+2*Float64(I₁==n₁) for I₁ in 1:n₁, I₂ in 1:n₂]
V=[]
for I₁ in 1:n₁, I₂ in 1:n₂
    if(I₁!=1&&I₁!=n₁)
        push!(V,[I₁,I₂])
    end
end

In [ ]:
# Example A-2
f(u)=0
@time C=[squa(NN,I₁,I₂,nip=20) for I₁ in 1:n₁, I₂ in 1:n₂];

g_coef=[Float64(I₂==1)*3+Float64(I₂==n₂) for I₁ in 1:n₁, I₂ in 1:n₂]
V=[]
for I₁ in 1:n₁, I₂ in 1:n₂
    if(I₂!=1&&I₂!=n₂)
        push!(V,[I₁,I₂])
    end
end

In [ ]:
# Example A-3
f(u)=4
@time C=[squa(NN,I₁,I₂,nip=20) for I₁ in 1:n₁, I₂ in 1:n₂];

g_coef=[0.0 for I₁ in 1:n₁, I₂ in 1:n₂]
V=[]
for I₁ in 1:n₁, I₂ in 1:n₂
    if(I₁!=1&&I₁!=n₁&&I₂!=1&&I₂!=n₂)
        push!(V,[I₁,I₂])
    end
end

In [ ]:
# Example B-1
f(u)=4
@time C=[squa(NN,I₁,I₂,nip=20) for I₁ in 1:n₁, I₂ in 1:n₂];

g_coef=[0.0 for I₁ in 1:n₁, I₂ in 1:n₂]
V=[]
for I₁ in 1:n₁, I₂ in 1:n₂
    if(I₁!=1&&I₁!=n₁&&I₂!=1&&I₂!=n₂)
        push!(V,[I₁,I₂])
    end
end

In [ ]:
# Solve Linear Equation
B̂=[sum(g_coef[I₁,I₂]*A[v1...,I₁,I₂] for I₁ in 1:n₁, I₂ in 1:n₂) for v1 in V]
Ĉ=[C[v1...] for v1 in V]
Â=[A[v1...,v2...] for v1 in V, v2 in V]
v̂=Â\(-B̂+Ĉ)
v_coef=zeros(n₁,n₂)
for i in 1:length(V)
    v_coef[V[i]...]=v̂[i]
end
u_coef=v_coef+g_coef

In [ ]:
# Makie Export
mesh=50,50
U=[[u₁,u₂] for u₁ ∈ range(NN.k[1][1],stop=NN.k[1][end],length=mesh[1]), u₂ ∈ range(NN.k[2][1],stop=NN.k[2][end],length=mesh[2])]
x=[NURBSmapping(NN,u)[1] for u in U]
y=[NURBSmapping(NN,u)[2] for u in U]
z=[sum(u_coef[I₁,I₂]*N(NN,I₁,I₂,u) for I₁ in 1:n₁, I₂ in 1:n₂) for u in U]
z0=[0 for u in U]
rand([-1f0, 1f0], 3)
pts = vec(Point3f0.(x, y, z))
Makie.surface(x, y, z)
Makie.surface!(x, y, z0,colormap = :Spectral)

In [ ]:
# Visualize Basis Functions
mesh=100,100
U=[[u₁,u₂] for u₁ ∈ range(NN.k[1][1],stop=NN.k[1][end],length=mesh[1]), u₂ ∈ range(NN.k[2][1],stop=NN.k[2][end],length=mesh[2])]
x=[NURBSmapping(NN,u)[1] for u in U]
y=[NURBSmapping(NN,u)[2] for u in U]
n₁,n₂=n=length.(NN.k)-(NN.p).-1
for I₁ in 1:n₁, I₂ in 1:n₂
    x=[NURBSmapping(NN,u)[1] for u in U]
    y=[NURBSmapping(NN,u)[2] for u in U]
    z=[N(NN,I₁,I₂,u) for u in U]
    global hoge=surface!(x, y, z)
end
hoge

In [ ]:
#Visualize Bi and Ni
mesh=30,50
U=[[u₁,u₂] for u₁ ∈ range(NN.k[1][1],stop=NN.k[1][end],length=mesh[1]), u₂ ∈ range(NN.k[2][1],stop=NN.k[2][end],length=mesh[2])]
x=[NURBSmapping(NN,u)[1] for u in U]
y=[NURBSmapping(NN,u)[2] for u in U]
z=[N(NN,1,2,u) for u in U]
u=[u[1] for u in U]
v=[u[2] for u in U]
w=[N(NN,1,2,u) for u in U]
surface(x, y, z)

p1 = surface(u, v, w)
p2 = surface(x, y, z)
scene = vbox(p1, p2)